In [1]:
try:
    import probml_utils as pml
except ModuleNotFoundError:
    %pip install -qq git+https://github.com/probml/probml-utils.git
    import probml_utils as pml

from probml_utils.dp_mixgauss_utils import NormalInverseWishart, gmm_gibbs, dp_mixgauss_gibbs

import jax.numpy as jnp
from jax import random, vmap
from jax.scipy.linalg import sqrtm
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'probml_utils.dp_mixgauss_utils'

In [ ]:
dim = 2
concentration = 1.0
num_of_clusters = 4
num_of_data = 300
num_of_samples = 1000

In [ ]:
# Example 1
key = random.PRNGKey(0)
# Generating the true data
key, *subkeys = random.split(key, 4)
niw_params1 = dict(loc=jnp.zeros(dim), mean_precision=0.02, df=dim + 5, scale=jnp.eye(dim))
niw1 = NormalInverseWishart(**niw_params1)
cluster_weights = random.dirichlet(subkeys[1], concentration * jnp.ones(num_of_clusters))
# cluster assignment of each datum
cluster_indices = random.categorical(subkeys[2], jnp.log(cluster_weights), shape=(num_of_data,))
cluster_params = niw1.sample(seed=subkeys[0], sample_shape=(num_of_clusters,))
cluster_covs = cluster_params["Sigma"]
cluster_means = cluster_params["mu"]
# data
mean_per_datum = cluster_means[cluster_indices]
cov_per_datum = cluster_covs[cluster_indices]
key, *subkeys = random.split(key, num_of_data + 1)
data1 = vmap(random.multivariate_normal, in_axes=(0, 0, 0))(jnp.array(subkeys), mean_per_datum, cov_per_datum)
# Clustering
gmm_samples1 = gmm_gibbs(key, num_of_samples, data1, concentration, num_of_clusters, niw_params1)
dp_samples1 = dp_mixgauss_gibbs(key, num_of_samples, data1, concentration, niw_params1)
example1 = (gmm_samples1[-1], dp_samples1[-1])

In [ ]:
# Example 2
key = random.PRNGKey(1)
# Generating the true data
key, *subkeys = random.split(key, 4)
niw_params2 = dict(loc=jnp.zeros(dim), mean_precision=0.05, df=dim + 5, scale=jnp.eye(dim))
niw2 = NormalInverseWishart(**niw_params2)
cluster_weights = random.dirichlet(subkeys[1], concentration * jnp.ones(num_of_clusters))
# cluster assignment of each datum
cluster_indices = random.categorical(subkeys[2], jnp.log(cluster_weights), shape=(num_of_data,))
cluster_params = niw2.sample(seed=subkeys[0], sample_shape=(num_of_clusters,))
cluster_covs = cluster_params["Sigma"]
cluster_means = cluster_params["mu"]
# data
mean_per_datum = cluster_means[cluster_indices]
cov_per_datum = cluster_covs[cluster_indices]
key, *subkeys = random.split(key, num_of_data + 1)
data2 = vmap(random.multivariate_normal, in_axes=(0, 0, 0))(jnp.array(subkeys), mean_per_datum, cov_per_datum)
# Clustering
gmm_samples2 = gmm_gibbs(key, num_of_samples, data2, concentration, num_of_clusters, niw_params2)
dp_samples2 = dp_mixgauss_gibbs(key, num_of_samples, data2, concentration, niw_params2)
example2 = (gmm_samples2[-1], dp_samples2[-1])

In [ ]:
# Example 3
key = random.PRNGKey(2)
# Generating the true data
key, *subkeys = random.split(key, 4)
niw_params3 = dict(loc=jnp.zeros(dim), mean_precision=0.5, df=dim + 5, scale=jnp.eye(dim))
niw3 = NormalInverseWishart(**niw_params3)
cluster_weights = random.dirichlet(subkeys[1], concentration * jnp.ones(num_of_clusters))
# cluster assignment of each datum
cluster_indices = random.categorical(subkeys[2], jnp.log(cluster_weights), shape=(num_of_data,))
cluster_params = niw3.sample(seed=subkeys[0], sample_shape=(num_of_clusters,))
cluster_covs = cluster_params["Sigma"]
cluster_means = cluster_params["mu"]
# data
mean_per_datum = cluster_means[cluster_indices]
cov_per_datum = cluster_covs[cluster_indices]
key, *subkeys = random.split(key, num_of_data + 1)
data3 = vmap(random.multivariate_normal, in_axes=(0, 0, 0))(jnp.array(subkeys), mean_per_datum, cov_per_datum)
# Clustering
gmm_samples3 = gmm_gibbs(key, num_of_samples, data3, concentration, num_of_clusters, niw_params3)
dp_samples3 = dp_mixgauss_gibbs(key, num_of_samples, data3, concentration, niw_params3)
example3 = (gmm_samples3[-1], dp_samples3[-1])

In [ ]:
# Plotting
cmap = plt.get_cmap("tab10")
data = (data1, data2, data3)
examples = (example1, example2, example3)
# Different rows represents different clustering results;
# different column represents different examples.
fig, axes = plt.subplots(2, 3)
plt.setp(axes, xticks=[], yticks=[])
for i in range(3):
    for j in range(2):
        samples = examples[i][j]
        clusters = jnp.unique(samples)
        for k in clusters:
            sub_samples = data[i][samples == k, :]
            axes[j, i].scatter(sub_samples[:, 0], sub_samples[:, 1], color=cmap(k))
plt.show()